<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function
%matplotlib inline

**Version**
Updated to work with v5.7.0 on 2020-05-21

# Storm Surge Tutorial - Katrina

This notebook is a tutorial that will walk you through setting up a storm surge simulation in GeoClaw.  This tutorial assumes you are relatively new to GeoClaw but will concentrate on the aspects of setup that pertain to storm surge rather than tsunamis.  Other notebooks and documentation that cover more of GeoClaw features can be found on the [Clawpack website](http://www.clawpack.org/gallery/notebooks.html#examples-using-geoclaw).

**Note:** this notebook is mostly intended to be a guide while working at the command-line.  To run the presented simulation run the command
```bash
make .plots
```
from the command line which should make the data files, compile the code and plot the results.

## Basics

We will start with a brief description of some of the background suggested for working with GeoClaw and storm surge simulations along with some nomenclature and deeper references.

### Mathematical/Physics Background

The adage "know enough to be dangerous" is more true here than we would like to admit.  If you do not know how to interpret your results you may be lead to false conclusions.  At the most basic you should understand that you are solving fluid flow governed by mass conservation and Newton's second law (momentum conservation with external forces).  It is also probably important to know that the *finite volume* method you are using is conservative when necessary. Check out the references below if you want more technical details on the equations, their derivation, and the methods used to solve these equations.

#### Equations
GeoClaw solves the shallow water equations in general, which for storm surge are written as
$$\begin{aligned}
    &h_t + (hu)_x + (hv)_y = 0 \\
    &(hu)_t + \left(hu^2 + \frac{1}{2} g h^2\right)_x + (huv)_y = fhv - ghb_x + \frac{h}{\rho}\left(-(P_A)_x + \rho_{\text{air}} C_w |\vec{W}| W_1 - C_f |\vec{u}| u \right ) \\
    &(hv)_t + (huv)_x + \left(hv^2 + \frac{1}{2} g h^2\right)_y = -fhu - ghb_y + \frac{h}{\rho} \left(-(P_A)_y + \rho_{\text{air}} C_w |W| W_2 - C_f |\vec{u}| v \right )
\end{aligned}$$
where
$$\begin{aligned}
    &h = \text{depth} &u,v = \text{depth averaged velocity components} \\
    &\rho = \text{density of water} &\rho_\text{air} = \text{density of air} \\
    &g = \text{gravitational acceleration} &b = \text{topography or bathymetry} \\
    &f = \text{Coriolis parameter} &\vec{W} = [W_1, W_2] = \text{wind velocity 10 m above the sea surface} \\
    &C_w = \text{wind friction coefficient} & C_f = \text{bottom friction coefficient} \\
\end{aligned}$$

The value of $C_w$ is a drag formula such as
$$
    C_w = \text{min}(2\times10^{-3}, (0.75 + 0.067 |W|) \times 10^{-3})
$$
and the value of the friction coefficient $C_f$ is determined using a hybrid Chezy-Manning's $n$ type friction law
$$
    C_f = \frac{g n^2}{h^{4/3}} \left[1-\left(\frac{h_{\text{break}}}{h}\right)^{\theta_f} \right]^{\gamma_f / \theta_f}
$$
where $n$ is the Manning's $n$ coefficient and $h_{\text{break}} = 2$, $\theta_f = 10$ and $\gamma_f = 4/3$ parameters control the form of the friction law.

### Technical Background

The technical background required to run GeoClaw is not tremendous.  The following leads you through an enumerated list with the most important background first following through to the most advanced least important.  If all you understand is programming in Python you are more than likely ok to get started.
 - **Python**, working in the *terminal* and with *git* are the most
   immediate technical skills you should have.  A good place to start are
   the lessons at https://software-carpentry.org/lessons/ along with a good
   google search.
 - **Makefiles** are used although it’s not as critical to understand how to write them.  If you are curious the above link also has a lesson.
 - **Fortran**, and to a lesser extent **C** and **C++** are used in the core part
   of the code.  If you want to modify one of the inner algorithms you will more than likely need these langauges.   There are a number of great tutorials online for any of these languages, even Fortran, with a bit of googling.
 - **High performance computing** and **parallel computing** knowledge is needed
   if you want to modify or contribute to the performance aspects of the code.  The
   training offered by [XSEDE](https://www.xsede.org/for-users/training) at https://www.xsede.org/for-users/training and via [Lawrence Livermore National Lab](https://hpc.llnl.gov/training/tutorials) at 
   https://hpc.llnl.gov/training/tutorials are highly recommended.

### Coding Nomenclature
 - **q**: This array contains the solution to the equations being solved.  Generally this is an array with components `[h, hu, hv]`, depth, momentum in the longitudinal and latitudinal directions respectively.  One quirk to watch for is that in the output there is an extra array element added on that gives the sea-surface `eta`.  This is done so that the results are easy to plot and the bathymetry/topography is easy to calculate.
 - **aux**: This array contains values that are generally placed at the same places as cells but are not updated by the core routines.  Examples of these are bathymetry, geometry of the sphere, friction, wind and pressure.
 - **gauges**: These are measurement points defined in space and time that observer one point and are used to mimic tide gauges among other things.
 - **levels**: Levels refer to the level of refinement.  A simulation that uses 3 levels of refinement would have a level at the coarsest resolution, one level at some refinement finer than the first, and a third at another finer level of refinement.
 

### References

These references describe [Clawpack](https://dx.doi.org/10.7717/peerj-cs.68), [GeoClaw](https://dx.doi.org/10.1016/j.advwatres.2011.02.016), [GeoClaw and storm surge](https://dx.doi.org/10.1016/j.ocemod.2014.01.002), and [the methods used more generally](http://depts.washington.edu/clawpack/book.html) respectively:
1. Mandli, K., Ahmadia, A., Berger, M., Calhoun, D., George, D., Hadjimichael,
   Y., Ketcheson, D., Lemoine, G., LeVeque, R. (2016). Clawpack: building an
   open source ecosystem for solving hyperbolic PDEs PeerJ Computer
   Science 2, e68. https://dx.doi.org/10.7717/peerj-cs.68
1. Berger, M., George, D., LeVeque, R., Mandli, K. (2011). The GeoClaw software
   for depth-averaged flows with adaptive refinement Advances in Water
   Resources 34, 1195 - 1206.
   https://dx.doi.org/10.1016/j.advwatres.2011.02.016
1. Mandli, K., Dawson, C. (2014). Adaptive Mesh Refinement for Storm Surge Ocean
   Modelling 75, 36 - 50. https://dx.doi.org/10.1016/j.ocemod.2014.01.002
1. LeVeque, R. (2002). Finite Volume Methods for Hyperbolic Problems.

## Setup

Setting up the simulation requires a number of inputs, most of which are common between all GeoClaw simulations.  The topics we will cover are:
1. setting the limits of the domain and the base (coarse) resolution,
1. controlling the simulation time and output,
1. adding gauges to measure output at points,
1. controlling the refinement,
1. adding topography/bathymetry,
1. setting variable friction, and
1. setting up the storm.

### Domain and Base Resolution

One of the first settings presented in the `setrun.py` file controls the size of the domain and coarsest resolution of the entire domain.  These lines look like
```python
# ---------------
# Spatial domain:
# ---------------

# Number of space dimensions:
clawdata.num_dim = num_dim

# Lower and upper edge of computational domain:
clawdata.lower[0] = -99.0      # west longitude
clawdata.upper[0] = -70.0      # east longitude

clawdata.lower[1] = 8.0       # south latitude
clawdata.upper[1] = 32.0      # north latitude

# Number of grid cells:
degree_factor = 4  # (0.25º,0.25º) ~ (25237.5 m, 27693.2 m) resolution
clawdata.num_cells[0] = int(clawdata.upper[0] - clawdata.lower[0]) * degree_factor
clawdata.num_cells[1] = int(clawdata.upper[1] - clawdata.lower[1]) * degree_factor
```
The first important lines are
```python
clawdata.lower[0] = -99.0      # west longitude
clawdata.upper[0] = -70.0      # east longitude

clawdata.lower[1] = 8.0       # south latitude
clawdata.upper[1] = 32.0      # north latitude
```
that control the longitudes and latitudes that mark the edges of the domain here we have a domain that spans [-99, -70] longitude from west to east and [8, 32] latitude from south to north.  Note that GeoClaw uses negative degrees west of the [IERS prime meridian](https://en.wikipedia.org/wiki/IERS_Reference_Meridian), which is slightly different than the more recognized [Greenwhich prime meridian](https://en.wikipedia.org/wiki/Prime_meridian_(Greenwich)).

The next setting of interest sets the number of cells in each direction to use to define the coarsest resolution:
```python
degree_factor = 4  # (0.25º,0.25º) ~ (25237.5 m, 27693.2 m) resolution
clawdata.num_cells[0] = int(clawdata.upper[0] - clawdata.lower[0]) * degree_factor
clawdata.num_cells[1] = int(clawdata.upper[1] - clawdata.lower[1]) * degree_factor
```
Here we have done a little bit of work to also allow specification of the resolution by number of cells per degree, here set to 4.

### Time

In GeoClaw the simulation time is arbitrary but it is useful to have some reference time.  Often for storms one of the landfall times is used.  If this is the case we will set have `t = 0` be the landfall time and reference everything relative to this time point.

One other thing of note, since GeoClaw counts in seconds but many of the times we set will be with respect to hours or days it is sometimes useful to define a helper function that converts between these.  One way to do this is
```python
# Time Conversions
def days2seconds(days):
    return days * 60.0**2 * 24.0
```

The relevant part of the `setrun.py` for the following settings is
```python
# -------------
# Initial time:
# -------------
clawdata.t0 = -days2seconds(3)

# Restart from checkpoint file of a previous run?
# If restarting, t0 above should be from original run, and the
# restart_file 'fort.chkNNNNN' specified below should be in
# the OUTDIR indicated in Makefile.

clawdata.restart = False               # True to restart from prior results
clawdata.restart_file = 'fort.chk00006'  # File to use for restart data

# -------------
# Output times:
# --------------

# Specify at what times the results should be written to fort.q files.
# Note that the time integration stops after the final output time.
# The solution at initial time t0 is always written in addition.

clawdata.output_style = 1

if clawdata.output_style == 1:
    # Output nout frames at equally spaced times up to tfinal:
    clawdata.tfinal = days2seconds(1)
    recurrence = 4
    clawdata.num_output_times = int((clawdata.tfinal - clawdata.t0) *
                                    recurrence / (60**2 * 24))

    clawdata.output_t0 = True  # output at initial (or restart) time?

elif clawdata.output_style == 2:
    # Specify a list of output times.
    clawdata.output_times = [0.5, 1.0]

elif clawdata.output_style == 3:
    # Output every iout timesteps with a total of ntot time steps:
    clawdata.output_step_interval = 1
    clawdata.total_steps = 1
    clawdata.output_t0 = True

clawdata.output_format = 'ascii'      # 'ascii' or 'binary'
clawdata.output_q_components = 'all'   # could be list such as [True,True]
clawdata.output_aux_components = 'all'
clawdata.output_aux_onlyonce = False    # output aux arrays only at t0
```

#### Initial Time
With that we are now ready to set some times in our simulation.  First is the time we want the simulation to begin.  Given that landfall is set to `t=0` we will want to start a few days before then to see how the hurricane progresses towards shore.  In this `setrun.py` we have set this to 3 days before landfall via
```python
clawdata.t0 = -days2seconds(3)
```

#### Output and Final Time
There are multiple output styles supported by GeoClaw but the one most often used is `output_style = 1`.  This allows us to specify a final time `tfinal` and the number of output we want between the initial time `t0` and `tfinal`.  The relevant code here is
```python
clawdata.tfinal = days2seconds(1)
recurrence = 4
clawdata.num_output_times = int((clawdata.tfinal - clawdata.t0) *
                                recurrence / (60**2 * 24))
```
We have set the final time for the simulation to be 1 day after landfall (again relative to landfall being `t=0`) and have asked for 4 output times per day.  Again you can simply specify a number for `num_output_times` but this allows you to specify easily by output per day.  For instance if you wanted to do hourly output you would do `recurrence = 24`.

#### Output Formats

The final piece here is the specification of what to output and the corresponding formatting.  The relevant code is
```python
clawdata.output_format = 'ascii'      # 'ascii' or 'binary'
clawdata.output_q_components = 'all'   # could be list such as [True,True]
clawdata.output_aux_components = 'all'
clawdata.output_aux_onlyonce = False    # output aux arrays only at t0
```
The first setting `output_format` allows you to output in either plain text ASCII, i.e. human readable text files, or a binary format.  The advantages of the binary format is that it leads to significantly smaller file sizes and are faster to write.

The second two settings we will ignore as they actually currently are not implemented (but you are welcome to do a pull requrest to fix that).

The fourth tells GeoClaw to only output the auxillary array once at the initial time or not.  Since we have time dependent auxillary arrays that contain the wind and pressure generally we need output at every output time.

### Gauges

Gauges allow for measurement of the values of either `q` or `aux` at place through a chosen time range.  To do this you can add gauges by appending to the gauges list via
```python

rundata.gaugedata.gauges.append([4, -94.13, 29.58,
                                 rundata.clawdata.t0,
                                 rundata.clawdata.tfinal])
```
This particular gauge is labeled as gauge "4", is located at longitude\latitude `(-94.13, 29.58)` and will be actively recording for the entire time of the simulation.  In addition to this you can specify what fields you would like to output.  As it is often useful to know the wind velocity and pressure we can add the line
```python
```
to record fields 4, 5, and 6 which are the x-velocity of wind, the y-velocity of wind and pressure in this particular case (more on determining which aux fields store what later).

**TODO:** Add references to docs.

### Refinement

There are a number of ways to specify refinement in GeoClaw.  The first important parameters are set by the code
```python
# max number of refinement levels:
amrdata.amr_levels_max = 2

# List of refinement ratios at each level (length at least amr_max_levels-1)
amrdata.refinement_ratios_x = [2, 4, 10]
amrdata.refinement_ratios_y = [2, 4, 10]
amrdata.refinement_ratios_t = [2, 4, 10]
```
Here we are allowing 2 levels, the coarsest level and one finer by setting `amr_levels_max = 2`.  The refinement ratios below this determine how much finer the next level is.  You should read these as being connective between each level so that the ratio 2 goes between level 1 (the coarsest) and level 2.  Correspondingly 4 is the refinement ratio between levels 2 and 3 and 10 between levels 3 and 4.  Note that we are not allowing this particular simulation to go above 2 even though we are specifying more ratios (they are just ignored).

#### Flagging
We have now defined how we will refine if asked but we need to determine when to refine, called flagging.  In GeoClaw we most often use direct values from the local cell to determine whether a cell needs to be flagged or not.  In the storm surge case there are two primary ways to do this:
 - **sea surface**: If this tolerance is exceeded by the sea surface then the cell is flagged.
 - **speed**: If this tolerance is exceeded by the magnitude of the velocity in the cell the cell is flagged.
 
The relevant code lines for setting these parameters are
```python
refine_data.wave_tolerance = 1.0
refine_data.speed_tolerance = [1.0, 2.0, 3.0, 4.0]
```
The first of these sets up the refinement criteria `wave_tolerance < h - sea_level`.  Note that `sea_level` is another important parameter near this spot in `setrun.py`.  The second is a bit more complex with tolerances between each levels such that `speed_tolereance[level] < sqrt(u(level)**2 + v(level)**2)` (note that some care is taken to remove a nearly dry cell for consideration).

One other important refinement criteria that limits flagging is the `deep_depth` and `max_level_deep` parameters.  These are set as
```python
refine_data.deep_depth = 300.0
refine_data.max_level_deep = 4
```
so that a cell that has a depth greater than 300 will not be refined past level 4 (although as we saw previously we are not allowing that anyway).

There are a number of other ways to control flagging as well such as
 - regions
 - storm characteristics
 - topography and bathymetry settings.

### Topography/Bathymetry

We also need to specify the topography and bathymetry to use for the simulation.  You can have as many files as you would like at different resolutions and that overlap.  To add a file simply do the following
```python
topo_path = os.path.join(scratch_dir, 'gulf_carribean.tt3')
topo_data.topofiles.append([3, 1, 5, rundata.clawdata.t0,
                                     rundata.clawdata.tfinal,
                                     topo_path])

```
The settings here describe the file, its location and how to treat it.  The first value describes the format of the file (3 is an ASCII file).  The next two numbers represent the resolution that the area the topography covers should be forced to.  Generally it is recomended to leave these to be any level you are considering and doing this instead by setting regions.  The next two are the times within when you want this file to be used.  The last `topo_path` is a path to this file.

### Friction

```python
# Variable friction
data.variable_friction = True

# Region based friction
# Entire domain
data.friction_regions.append([rundata.clawdata.lower,
                              rundata.clawdata.upper,
                              [np.infty, 0.0, -np.infty],
                              [0.030, 0.022]])

# La-Tex Shelf
data.friction_regions.append([(-98, 25.25), (-90, 30),
                              [np.infty, -10.0, -200.0, -np.infty],
                              [0.030, 0.012, 0.022]])

```

Variable friction can be important to accurately simulated overland flooding such as in a storm surge event.  In GeoClaw we compute friction impacts via a Manning's N formulation which takes the form
$$
hu \frac{g n^2}{h^{4/3}} \sqrt{u^2 + v^2}
$$
where the parameter $n$ represents a form of roughness of the bottom surface.  GeoClaw currently allows partial support for variable spatial specification of the parameter $n$ by specifying regions and iso-baths.  Take a look again at the `setrun.py` file where right after the function `set_storm` we have the function `set_friction`.

The first parameter turns on variable friction via
```python
data.variable_friction = True
```
Next are two regions that are specified.  The format of the list takes on the form of the region that the rule specified covers, the iso-baths between which the friction will be set, and finally the actual coefficients themselves.

The first is
```python
data.friction_regions.append([rundata.clawdata.lower,
                              rundata.clawdata.upper,
                              [np.infty, 0.0, -np.infty],
                              [0.030, 0.022]])
```
and sets a variable friction region that covers the entire domain and sets the friction to `0.030` below bathymetry/topography at `0.0` and to 0.022 to regions above `0.0`.  The second
```python
data.friction_regions.append([(-98, 25.25), (-90, 30),
                              [np.infty, -10.0, -200.0, -np.infty],
                              [0.030, 0.012, 0.022]])
```
only covers the region between longitudes (-98, -90) and latitudes (25.25, 30).  Here the friction is set to `0.03` above -10 meters, to `0.012` between -10 and -200 meters, and to `0.022` below -200 meters.  This latter rule was specified to represent the abnormally smooth Louisiana/Texas shelf region.



### The Storm

Finally we come to setting up the storm.  First we will discuss the `storm` class that allows us to read in and manipulate storm data and then describe how we might use it in a `setrun.py` file.

#### The Storm Class

**TODO:** Add description.

#### Example
We will cover one of the ways to do this from an ATCF formatted file from NOAA's database of best-track storm files.  You can of course modify this however you want.  The relevant code we will discuss in this section is
```python
# Source term controls
data.wind_forcing = True
data.pressure_forcing = True
data.drag_law = 1

data.display_landfall_time = True

# AMR parameters, m/s and m respectively
data.wind_refine = [20.0, 40.0, 60.0]
data.R_refine = [60.0e3, 40e3, 20e3]

# Storm parameters - Parameterized storm (Holland 1980)
data.storm_specification_type = 'holland80'  # (type 1)
data.storm_file = os.path.expandvars(os.path.join(os.getcwd(),
                                     'ike.storm'))

# Convert ATCF data to GeoClaw format
clawutil.data.get_remote_file(
               "http://ftp.nhc.noaa.gov/atcf/archive/2008/bal092008.dat.gz")
atcf_path = os.path.join(scratch_dir, "bal092008.dat")
# Note that the get_remote_file function does not support gzip files which
# are not also tar files.  The following code handles this
with gzip.open(".".join((atcf_path, 'gz')), 'rb') as atcf_file,    \
        open(atcf_path, 'w') as atcf_unzipped_file:
    atcf_unzipped_file.write(atcf_file.read().decode('ascii'))

# Read in the newly downloaded and decompressed file
ike = Storm(path=atcf_path, file_format="ATCF")

# Calculate landfall time - Need to specify as the file above does not
# include this info (9/13/2008 ~ 7 UTC)
ike.time_offset = datetime.datetime(2008, 9, 13, 7)

ike.write(data.storm_file, file_format='geoclaw')
```

#### Forcing
```python
# Source term controls
data.wind_forcing = True
data.pressure_forcing = True
data.drag_law = 1
```
The first two parameters above turn on and off wind and pressure forcing respectively.  The third describes what kind of wind friction drag law to use.  These have the form
$$
    \rho_{\text{air}} C_w |W| W
$$
where $W$ is the wind velocity.  The determination of the friction coefficient $C_w$ is determined by the parameter `drag_law` for which valid values can be 
 - `0` for no drag,
 - `1` for Garret wind drag, and
 - `2` for Powell wind drag.
The Garret wind drag takes the form
$$
    C_w = \min(W_\text{limit}, 0.75 + 0.067 + W) \times 10^{-3}
$$
where as Powell uses a similar approach but is sector based as detailed in

M.D. Powell (2006). “Final Report to the National Oceanic and 
  Atmospheric Administration (NOAA) Joint Hurricane Testbed (JHT) 
  Program.” 26 pp.

#### Progress Display
```python
data.display_landfall_time = True
```
This setting simply changes the time displayed in the console so that it reads relative to landfall and days.

#### Storm Refinement Criteria
```python
# AMR parameters, m/s and m respectively
data.wind_refine = [20.0, 40.0, 60.0]
data.R_refine = [60.0e3, 40e3, 20e3]
```
These parameters are similar to the speed parameter above and flag cells.  The first parameter controls refinement based on wind speed in m/s while the second is based on distance to the eye of the storm.  Each of these are specified as a Python list such that refinement is forced to the next level if it surpasses the given parameter.  For instance if you are within 60 kilometers of the storm's eye refinement is forced to level, 40 kilometers is forced to level 3, and within 20 kilometers to level 4. 

#### Storm Field Parameterization and Data Location
```python
# Storm parameters - Parameterized storm (Holland 1980)
data.storm_specification_type = 'holland80'  # (type 1)
data.storm_file = os.path.expandvars(os.path.join(os.getcwd(),
                                     'ike.storm'))
```
These parameters specify what kind of storm field parameterization to use.  In this case we will use the paramterization specified in Holland's 1980 paper.  The second parameter tells GeoClaw where to look for the storm data such as the track, intensity, pressure, etc.  This file should be formatted in the GeoClaw storm format.

#### Download ATCF Data from NOAA
```python
# Convert ATCF data to GeoClaw format
clawutil.data.get_remote_file(
               "http://ftp.nhc.noaa.gov/atcf/archive/2008/bal092008.dat.gz")
atcf_path = os.path.join(scratch_dir, "bal092008.dat")
# Note that the get_remote_file function does not support gzip files which
# are not also tar files.  The following code handles this
with gzip.open(".".join((atcf_path, 'gz')), 'rb') as atcf_file,    \
        open(atcf_path, 'w') as atcf_unzipped_file:
    atcf_unzipped_file.write(atcf_file.read().decode('ascii'))
```
This code fetches the data for the storm we are interested in fetching, which is located at http://ftp.nhc.noaa.gov/atcf/archive/2008/bal092008.dat.gz. Note that this is a great place to get data to use for most storms througout the world.

#### Read in the ATCF Storm Data into a Storm Object
```python
# Read in the newly downloaded and decompressed file
ike = Storm(path=atcf_path, file_format="ATCF")
```
Next we read in the uncompressed ATCF data and create a storm object.

#### Set the Landfall Time
```python

# Calculate landfall time - Need to specify as the file above does not
# include this info (9/13/2008 ~ 7 UTC)
ike.time_offset = datetime.datetime(2008, 9, 13, 7)
```
Now we set a time offset so that we know relative to the data in the ATCF file what time is `t = 0`.  Again this is often set to landfall but in the end this parameter does not matter except to know where we are relative to the original ATCF dates.

#### Write Out the Storm Object
```python
ike.write(data.storm_file, file_format='geoclaw')
```
Finally we write out the new storm data into the GeoClaw storm format. We are now ready to have GeoClaw read it in and use the data.

## Running

**TODO:** Add more description here

In [ ]:
%%bash
make .data
make .output

## Plotting

There are a couple of storm surge specific plotting options that are important to note.  Opening the `setplot.py` file we can observe some of these.  If you want to generate the plots run `make .plots` from the command line after running a simulation.

#### Surge Plotting Module

A number of helpful functions that help to plot storm surge simulations is provided in the module `clawpack.geoclaw.surge.plot` found at the path `$CLAW/src/python/geoclaw/surge/plot.py`.  We will cover a couple of relevant functions in this module but leave many of them up to the interested reader.

First off at the top of the `setplot` file you will note the definition of the function `surge_afteraxes` via
```python
surge_afteraxes = lambda cd: surge.surge_afteraxes(cd, track, landfall, plot_direction=False)
```
which is used for most of the plots produced.  This function plots the location of the eye of the storm, the direction of the storm, and modifies the title to days relative to landfall.  The reason the function is redefined in the `setplot` file is that we want to hand it the track of the storm which is read in at the top of the `setplot` file.

Additionally the surge plotting module contains a number of helper functions that create `plotitem`s including surface elevation, land, water speed, and friction.  Looking at the first plot description for the surface you will notice the three lines
```python
surge.add_surface_elevation(plotaxes, bounds=surface_limits, shrink=full_shrink)
surge.add_land(plotaxes, topo_min=-10.0, topo_max=5.0)
surge.add_bathy_contours(plotaxes)
```
which add plot items to plot the surface elevation, land, and a bathymetry contour respectively.

Along with the solution plotting you can also plot the wind speed and pressure via the functions
```python
surge.add_wind(plotaxes, bounds=wind_limits, plot_type='imshow')
surge.add_pressure(plotaxes, bounds=pressure_limits)
```
respectively.

## Final Workflow
Since the most common setup for running GeoClaw is not from a notebook we will describe this final workflow as if we were in the command line (although the following can be run from here).

1. Edit `setrun.py` to setup the simulation paramters
1. Edit `setplot.py` to setup the plotting you want
1. Run `make` from the command-line.

In [ ]:
%%bash
make .plots